# How To Create Reference Files in ASDF Format

### Nadia Dencheva, SSB
#### Mar 1st, 2016

This tutorial explains how to create reference files in ASDF format for the JWST pipeline.
It will focus on WCS reference files, however, ASDF can be used for other reference data as well, especially those involving models, as they are not easily representable in FITS.

### Outline

- Quick Introduction
- Modeling Capabilities
  - Single Models
  - Combined Models
  - Inverse of a model
- WCS Reference Files 


### What is ASDF?

ASDF is a human-readable, hierarchical metadata structure, made up of basic dynamic data types such as strings, numbers, lists and mappings. Data is saved as binary arrays. It is primarily intended as an interchange format for delivering products from instruments to scientists or between scientists. It’s based on YAML and JSON schema and as such provides automatic structure and metadata validation.

Advantages of using ASDF for WCS reference files:

- Provides uniform representation of transforms in the JWST pipeline.
- Provides automatic validation.

### Quick Introduction

ASDF provides an easy way to save transforms to a file. The simplest way to do this is to create the transform using astropy.modeling and then use the asdf package to save it to a file.

In [90]:
import numpy as np
from astropy.modeling import models
from asdf import AsdfFile

Create a 1D Polynomial model.

In [91]:
poly1 = models.Polynomial1D(degree=3, c0=.1, c1=.2, c2=.3, c3=.4)
print poly1

Model: Polynomial1D
Inputs: (u'x',)
Outputs: (u'y',)
Model set size: 1
Degree: 3
Parameters:
     c0  c1  c2  c3
    --- --- --- ---
    0.1 0.2 0.3 0.4


In [93]:
x = np.arange(5)
z = poly1(x)
print(z)

[  0.1   1.    4.9  14.2  31.3]


Create an AsdfFile object and add the model to its tree. An Asdf file has a dictionary, called `tree`.

In [96]:
f = AsdfFile()
f.tree['model'] = poly1
print(f.tree)


{'model': <Polynomial1D(3, c0=0.1, c1=0.2, c2=0.3, c3=0.4)>}


Saving it to a file validates it.

In [99]:
f.write_to('poly.asdf')
f.write_to('poly.asdf', all_array_storage='inline')
f.write_to('poly.asdf', auto_inline=10)
!less poly.asdf

#ASDF 1.0.0
#ASDF_STANDARD 1.0.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.0.0
asdf_library: !core/software-1.0.0 {author: Space Telescope Science Institute, homepage: 'http://github.com/spacetelescope/pyasdf',
  name: asdf, version: 1.1.0.dev715}
model: !transform/polynomial-1.0.0
  coefficients: !core/ndarray-1.0.0
    data: [0.1, 0.2, 0.3, 0.4]
    datatype: float64
    shape: [4]
...


To create a model from an asdf file, just open the file and acess the model.

In [102]:
fa = AsdfFile.open('poly.asdf')
model = fa.tree['model']
tree = fa.tree
print(tree.keys())
#print(model)

[u'model', u'asdf_library']


In [103]:
model(x)

array([  0.1,   1. ,   4.9,  14.2,  31.3])

### Modeling capabilities

In [104]:
from astropy.modeling import models
#models.

Examples of models often used with WCS transformations.

In [105]:
offset = models.Shift(-2048)
scale = models.Scale(8.301)

There are several rotation models.

Do not set the translation in an affine transform to define a rotation using a rotation matrix.

In [106]:
angle = np.deg2rad(12.1)
mat = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
affine = models.AffineTransformation2D(matrix=mat, translation=[1.1, 2.2])

This defines a rotation by an angle. The angle is in degrees and the direction is counter-clockwise.

In [107]:
rotation = models.Rotation2D(angle=21.7)
print(rotation(1,2))

(0.1896390569863976, 2.2280119003419414)


Several polynomial models are defined as well, including Chebyshev and Legendre polynomials.

In [109]:
poly2 = models.Polynomial2D(degree=1, c0_0=.1, c0_1=.2, c1_0=.3)
print(poly2(1, 2))

0.8


#### Combining Models


- **Composition**: evaluating models in series. 

The output of one model is passed as input to the next one. The composition operator is `|`.


In [111]:
model = offset | scale
model(1) == scale(offset(1))


True

The number of outputs of a model must equal the number of inputs of the next model.

In [112]:
# This raises an erros
model = offset | rotation


ModelDefinitionError: Unsupported operands for |: None (n_inputs=1, n_outputs=1) and None (n_inputs=2, n_outputs=2); n_outputs for the left-hand model must match n_inputs for the right-hand model.

- Model **concatenation**  - evaluating models on independent variables and concatenating the outputs. The inputs are distributed to all models. The number of inputs must equal the total number of inputs of all models.

In [113]:
poly_x = models.Polynomial2D(degree=1, c0_0=.1, c0_1=.2, c1_0=.3, name='poly_x')
poly_y = models.Polynomial2D(degree=1, c0_0=.4, c0_1=.5, c1_0=.6, name='poly_y')
model = poly_x & poly_y

In [114]:
model(1, 2)

TypeError: __call__() takes at least 5 arguments (3 given)

In [115]:
model.n_inputs

4

Two models in astropy.modeling are designed to manage the inputs.

`Mapping` is used to change the order of inputs or drop inputs.

`Identity` is used to pass inputs unchanged. It is typically used with the join operator `&`.

In [116]:
map_inputs = models.Mapping((0, 1, 0, 1), name='map_poly_inputs')
distortion = map_inputs | model
distortion(1, 2)

(0.8, 2.0)

In [117]:
f = AsdfFile()
f.tree['model'] = distortion
f.write_to('distortion.asdf', all_array_storage='inline')
!less distortion.asdf

#ASDF 1.0.0
#ASDF_STANDARD 1.0.0
%YAML 1.1
%TAG ! tag:stsci.edu:asdf/
--- !core/asdf-1.0.0
asdf_library: !core/software-1.0.0 {author: Space Telescope Science Institute, homepage: 'http://github.com/spacetelescope/pyasdf',
  name: asdf, version: 1.1.0.dev715}
model: !transform/compose-1.0.0
  forward:
  - !transform/remap_axes-1.0.0
    mapping: [0, 1, 0, 1]
    name: map_poly_inputs
  - !transform/concatenate-1.0.0
    forward:
    - !transform/polynomial-1.0.0
      coefficients: !core/ndarray-1.0.0
        data:
        - [0.1, 0.2]
        - [0.3, 0.0]
        datatype: float64
        shape: [2, 2]
      name: poly_x
    - !transform/polynomial-1.0.0
      coefficients: !core/ndarray-1.0.0
        data:
        - [0.4, 0.5]
        - [0.6, 0.0]
        datatype: float64
        shape: [2, 2]
      name: poly_y
...


- **Arithmetic operations** with models. The same inputs are passed to all models.

In [118]:
model = poly_x + poly_y
print(model.n_inputs)

2


In [119]:
print(model(1, 2))
print(poly_x(1, 2) + poly_y(1, 2))

2.8
2.8


If a model has an analytical innverse, it is already defined. It can be accesses through the `inverse` property.

In [120]:
print(offset)

Model: Shift
Inputs: (u'x',)
Outputs: (u'x',)
Model set size: 1
Parameters:
     offset
    -------
    -2048.0


In [121]:
print(offset.inverse)

Model: Shift
Inputs: (u'x',)
Outputs: (u'x',)
Model set size: 1
Parameters:
    offset
    ------
    2048.0


An inverse can be assigned or rest by assigning to the `inverse` property.

In [122]:
poly1.inverse

NotImplementedError: An analytical inverse transform has not been implemented for this model.

In [123]:
poly1.inverse = models.Polynomial1D(degree=1, c0=10, c1=1)
print(poly1.inverse)

Model: Polynomial1D
Inputs: (u'x',)
Outputs: (u'y',)
Model set size: 1
Degree: 1
Parameters:
     c0   c1
    ---- ---
    10.0 1.0


The inverse property can be used with combined models as well. Tking the `distortion` model above as an example:

In [124]:
print(distortion)

Model: CompoundModel71
Inputs: ('x0', 'x1')
Outputs: (u'z0', u'z1')
Model set size: 1
Parameters:
    c0_0_1 c1_0_1 c0_1_1 c0_0_2 c1_0_2 c0_1_2
    ------ ------ ------ ------ ------ ------
       0.1    0.3    0.2    0.4    0.6    0.5


In [125]:
print(distortion.submodel_names)
print(distortion.n_outputs)
print(distortion.n_inputs)

('map_poly_inputs', 'poly_x', 'poly_y')
2
2


In [126]:
print(distortion.inverse)

NotImplementedError: All models in a composite model must have an inverse defined in order for the composite model to have an inverse.  <Polynomial2D(1, c0_0=0.1, c1_0=0.3, c0_1=0.2, name='poly_x')> does not have an inverse.

In [127]:
dist_inverse = map_inputs | poly_y & poly_x
distortion.inverse = dist_inverse
print(distortion.inverse)

Model: CompoundModel74
Inputs: ('x0', 'x1')
Outputs: (u'z0', u'z1')
Model set size: 1
Parameters:
    c0_0_1 c1_0_1 c0_1_1 c0_0_2 c1_0_2 c0_1_2
    ------ ------ ------ ------ ------ ------
       0.4    0.6    0.5    0.1    0.3    0.2


### JWST Reference Files

#### List of reference types used by assign_wcs

| reftype             |                        description                | Instrument
|---------------------|---------------------------------------------------|-----------
| **camera**          | NIRSPEC Camera model                              | Nirspec
| **collimator**      | NIRSPEC Collimator Model                          | Nirspec
| **disperser**       | Disperser parameters                              | Nirspec
| **distortion**      | Spatial distortion model                          | MIRI, Nircam
| **filteroffset**    | MIRI Imager fiter offsets                         | Nirspec
| **fore**            | Transform through the NIRSPEC FORE optics         | Nirspec
| **fpa**             | Transform in the NIRSPEC FPA plane                | Nirspec
| **msa**             | Transform in the NIRSPEC MSA plane                | Nirspec
| **ote**             | Transform through the Optical Telescope Element   | Nirspec
| **specwcs**         | Wavelength calibration models                     | MIRI LRS, MRS
| **regions**         | Stores location of the regions on the detector    | MIRI MRS
| **v2v3**            | Transform from MIRI focal plane to V2V3 plane     | MIRI MRS
| **wavelengthrange** | Typical wavelength ranges                         | MIRI, Nirspec

More information about the reference files in Build 5, including the CRDS rules is here:

http://ssb.stsci.edu/doc/jwst_dev/jwst_pipeline.assign_wcs.doc/html/reference_files.html

#### Conventions used in the JWST Pipeline

- Forward transform is from detector to world coordinates.
- The transforms in the pipeline are 0-based.
- The center of the pixel is an integer index, i.e. pixel 1 is from 0.5 to 1.5.

Scripts used for creating Build 5 reference files are in 

https://aeon.stsci.edu/ssb/trac/jwst/browser/trunk/jwst_pipeline/assign_wcs/jwst_pipeline/assign_wcs/tools

#### An example Nirspec camera reference file.

In [129]:
#!less Camera.pcf


In [ ]:
def coeffs_from_pcf(degree, coeffslist):
    coeffs = {}
    k = 0
    for i in range(degree + 1):
        for j in range(degree + 1):
            if i+j < degree+1:
                name = "c{0}_{1}".format(i, j)
                coeffs[name] =  float(coeffslist[k])
                k +=1
            else:
                continue
    return coeffs

In [ ]:
 with open('Camera.pcf') as f:
        lines = [l.strip() for l in f.readlines()]

# Note that all Nirspec transforms are defined from sky to detector.
# Their inverse will be the forward transform in the pipeline.
factors = lines[lines.index('*Factor 2') + 1].split()
scale = models.Scale(1 / float(factors[0]), name="x_scale") & \
        models.Scale(1 / float(factors[1]), name="y_scale")

rotation_angle = lines[lines.index('*Rotation') + 1]
# Backward rotation is in the counter-clockwise direction as in modeling
# Forward is clockwise
backward_rotation = models.Rotation2D(float(rotation_angle), name='rotation')
rotation = backward_rotation.copy()

# Here the model is called "output_shift" but in the team version it is the "input_shift".
input_rot_center = lines[lines.index('*InputRotationCentre 2') + 1].split()
output_offset = models.Shift(float(input_rot_center[0]), name='output_x_shift') & \
             models.Shift(float(input_rot_center[1]), name='output_y_shift')

# Here the model is called "input_shift" but in the team version it is the "output_shift".
output_rot_center = lines[lines.index('*OutputRotationCentre 2') + 1].split()
input_offset = models.Shift(-float(output_rot_center[0]), name='input_x_shift') & \
              models.Shift(-float(output_rot_center[1]), name='input_y_shift')

degree = int(lines[lines.index('*FitOrder') + 1])

xcoeff_index = lines.index('*xForwardCoefficients 21 2')
xlines = lines[xcoeff_index + 1: xcoeff_index + 22]
xcoeff_forward = coeffs_from_pcf(degree, xlines)
x_poly_backward = models.Polynomial2D(degree, name='x_poly_backward', **xcoeff_forward)

ycoeff_index = lines.index('*yForwardCoefficients 21 2')
ycoeff_forward = coeffs_from_pcf(degree, lines[ycoeff_index + 1: ycoeff_index + 22])
y_poly_backward = models.Polynomial2D(degree, name='y_poly_backward', **ycoeff_forward)

xcoeff_index = lines.index('*xBackwardCoefficients 21 2')
xcoeff_backward = coeffs_from_pcf(degree, lines[xcoeff_index + 1: xcoeff_index + 22])
x_poly_forward = models.Polynomial2D(degree, name='x_poly_forward', **xcoeff_backward)

ycoeff_index = lines.index('*yBackwardCoefficients 21 2')
ycoeff_backward = coeffs_from_pcf(degree, lines[ycoeff_index + 1: ycoeff_index + 22])
y_poly_forward = models.Polynomial2D(degree, name='y_poly_forward', **ycoeff_backward)

x_poly_forward.inverse = x_poly_backward
y_poly_forward.inverse = y_poly_backward

output2poly_mapping = models.Identity(2, name='output_mapping')
output2poly_mapping.inverse = models.Mapping([0, 1, 0, 1])
input2poly_mapping = models.Mapping([0, 1, 0, 1], name='input_mapping')
input2poly_mapping.inverse = models.Identity(2)

model_poly = input2poly_mapping | (x_poly_forward & y_poly_forward) | output2poly_mapping

model = model_poly | input_offset |scale |rotation |output_offset

f = AsdfFile()
#f.tree = ref_file_kw.copy()
f.tree['model'] = model
f.write_to('camera.asdf')

In [ ]:
#!less camera.asdf

#### Where should we keep the scripts?

- a repository in github

  https://github.com/spacetelescope
  
  
- SVN